<a href="https://colab.research.google.com/github/kanisshka23/Deep-learning-/blob/main/Handwriting_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install easyocr transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 10.3 MB/s eta 0:00:00


In [ ]:
import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load a pre-trained NER model (fine-tuned on token classification)
model_name = '/content/drive/MyDrive/finetunedner'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import easyocr
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

image_path = '/content/sampleprescription.png'
# Function to extract text from an image using EasyOCR
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['en'])  # Initialize the reader for English
    results = reader.readtext(image_path, detail=0)  # Extract text without bounding box details
    extracted_text = ' '.join(results)  # Join the recognized words
    return extracted_text

# Load the pre-trained BioBERT model and tokenizer
model_name = "/content/drive/MyDrive/biobert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create a pipeline for Named Entity Recognition (NER)
nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Function to classify medical terms in text
def classify_medical_terms(text):
    ner_results = nlp(text)
    classified_terms = {
        "DRUG": [],
        "DOSAGE": [],
        "DURATION": [],
        "TIME": [],
        "PATIENT": []
    }

    for entity in ner_results:
        entity_label = entity['entity_group'].upper()

        if "DRUG" in entity_label:
            classified_terms["DRUG"].append(entity['word'])
        elif "DOSE" in entity_label or "DOSAGE" in entity_label:
            classified_terms["DOSAGE"].append(entity['word'])
        elif "DURATION" in entity_label:
            classified_terms["DURATION"].append(entity['word'])
        elif "TIME" in entity_label:
            classified_terms["TIME"].append(entity['word'])
        elif "PATIENT" in entity_label:
            classified_terms["PATIENT"].append(entity['word'])

    return classified_terms

# Main function to run the pipeline
def process_medical_prescription(image_path):
    # Step 1: Extract text from image using EasyOCR
    extracted_text = extract_text_from_image(image_path)
    print(f"Extracted Text: {extracted_text}")

    # Step 2: Classify medical terms using BioBERT
    classified_terms = classify_medical_terms(extracted_text)

    return classified_terms

# Example usage:
image_path = '/content/sampleprescription.png'  # Replace with your image path
classified_data = process_medical_prescription(image_path)
print(f"Classified Medical Terms: {classified_data}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Extracted Text: DD FORM  1289 NOv 71 DOD PRESCRIPTION FOR (Full namo, address, & phone numbar)  (H under12, givo 3g0) Jebn&_Dee _ HMB USN Us.S Neverfergotten (DD 78) MEDICAL FACLIY DATE USS Never Forgotten (DD !78) 23 Jan 49 R (Superscription) gm Or ml: (Inscription) Ksebdanna 15 me Ampheaee 120 me (Subscription) mv #fL fuetwn> (Signe) Sm2 Ld 0.€ MFG: Wzeth  EXP DATE: 1loz LOTNO: P3ak06 FLLED BY: KMt Jack Z Frost LCDE ID, USIR 10072 R NUmbER SKGNATURE RANK AnO OEGREE EOMION OF JAN 80 MAY Be USEd  For Sin 0102-LF-012-6201 3984 Sv'
Classified Medical Terms: {'DRUG': [], 'DOSAGE': [], 'DURATION': [], 'TIME': [], 'PATIENT': []}
